In [ ]:
pip install opencv-contrib-python

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
%matplotlib inline

In [ ]:
def keypointsAndDescriptors(img):
    sift = cv2.xfeatures2d.SIFT_create()
    return sift.detectAndCompute(img, None)

def L1_norm(x, y):
    return np.linalg.norm((x - y), ord=1)

def ownMatcher(descriptor1, descriptor2):
    matches = []

    for i, el1 in enumerate(descriptor1):
        for j, el2 in enumerate(descriptor2):
            matches.append(
                cv2.DMatch(
                    _distance=L1_norm(el1, el2),
                    _imgIdx=0,
                    _queryIdx=i,
                    _trainIdx=j
                )
            )

    return matches

In [ ]:
def drawMatcher(image1, image2, descriptors1, descriptors2, keypoints1, keypoints2, matches, title):
    plt.figure(figsize=(25, 15))
    plt.axis('off')
    match_img = cv2.drawMatches(image1, keypoints1, image2, keypoints2, matches, None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
    plt.title(f'{title}', fontsize=30)
    plt.axis('off')
    plt.imshow(match_img)
    plt.show()

In [ ]:
def compare_matchers(des1, des2):
    start = datetime.now()
    matches12_own = ownMatcher(des1, des2)
    print("Custom Brute Force matcher took:", datetime.now() - start)

    start = datetime.now()
    bf = cv2.BFMatcher(cv2.NORM_L1, crossCheck=True)
    matches12_bf = bf.match(des1, des2)
    print("CV2 Brute Force matcher took:", datetime.now() - start)

    return matches12_own, matches12_bf

In [ ]:
def pipeline(path1, path2):
    img1 = cv2.imread(path1)
    # Please do note that SIFT operates only on grayscale images anyway,
    # and that the default OpenCV implementation converts images to grayscale anyway so we don't have to.
    img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2RGB)

    img2 = cv2.imread(path2)
    img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2RGB)

    figure, ax = plt.subplots(1, 2, figsize=(16, 8))

    ax[0].imshow(img1)
    ax[1].imshow(img2)

    kp1, des1 = keypointsAndDescriptors(img1)
    kp2, des2 = keypointsAndDescriptors(img2)

    matches12_own, matches12_bf = compare_matchers(des1, des2)

    best_20_matches12_own = sorted(matches12_own, key=lambda x: x.distance)
    best_20_matches12_own = best_20_matches12_own[:20]

    best_20_matches12_bf = sorted(matches12_bf, key=lambda x: x.distance)
    best_20_matches12_bf = best_20_matches12_bf[:20]

    drawMatcher(img1, img2, des1, des2, kp1, kp2, best_20_matches12_own, "Custom Brute Force Matcher (20 matches)")
    drawMatcher(img1, img2, des1, des2, kp1, kp2, best_20_matches12_bf, "CV2 Brute Force Matcher (20 matches)")
    
    drawMatcher(img1, img2, des1, des2, kp1, kp2, matches12_own, "Custom Brute Force Matcher (all matches)")
    drawMatcher(img1, img2, des1, des2, kp1, kp2, matches12_bf, "CV2 Brute Force Matcher (all matches)")

    return des1, des2, matches12_own, matches12_bf

In [ ]:
des3, des4, matches34_own, matches34_bf = pipeline("Image2.jpg", "Mutated2.jpg")

In [ ]:
des1, des2, matches12_own, matches12_bf = pipeline("Image1.jpg", "Mutated1.jpg")

In [ ]:
# Magic, I ain't gotta explain it
# (eyeballing)
MAGIC_NUMBER = 20

In [ ]:
def choose_n_best(m, n):
    return sorted(m, key=lambda x: x.distance)[:n]

In [ ]:
def matcher_stats(m):
    m = choose_n_best(m, MAGIC_NUMBER)

    dists = np.array([match.distance for match in m])
    print("Average:", np.average(dists))
    print("Standard Deviation:", np.std(dists))

    plt.bar(range(len(m)), dists, width=1.0)
    plt.show()

In [ ]:
def stat_pipeline(m1, m2):
    print("Total Match Amount")
    print("Custom:", len(m1), "CV2:", len(m2))
    print()
    print("Custom BF Stats")
    matcher_stats(m1)
    print("CV2 BF Stats")
    matcher_stats(m2)
    print()
    print("Comparison")
    print("Total Absolute Difference",
        sum(np.array(
            [match.distance for match in choose_n_best(m1, MAGIC_NUMBER)]
        ) - np.array(
            [match.distance for match in choose_n_best(m2, MAGIC_NUMBER)]
        ))
    )

In [ ]:
stat_pipeline(matches34_own, matches34_bf)

In [ ]:
stat_pipeline(matches12_own, matches12_bf)

In [ ]:
# Looks like a cumulative from normal distribution